# XPRESSpipe methodology analysis

Copyright (C) 2019  Jordan A. Berg   
jordan.berg@biochem.utah.edu   
   
This program is free software: you can redistribute it and/or modify it under
the terms of the GNU General Public License as published by the Free Software
Foundation, either version 3 of the License, or (at your option) any later
version.    
    
This program is distributed in the hope that it will be useful, but WITHOUT ANY
WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A
PARTICULAR PURPOSE. See the GNU General Public License for more details.    
    
You should have received a copy of the GNU General Public License along with
this program.  If not, see <https://www.gnu.org/licenses/>.

### Import dependencies

In [ ]:
from __future__ import print_function
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
matplotlib.rcParams['font.sans-serif'] = 'Arial'
from matplotlib_venn import venn2, venn3
import seaborn as sns
from xpressplot import convert_names, rpm, check_samples, threshold
from scipy import stats
from statsmodels.stats.multitest import multipletests
%matplotlib inline
import random
import ast
from urllib.request import Request, urlopen
import plotly
import plotly.offline as py
import plotly_express as px

### Set global file names and directories
- Make sure the above dependencies have been installed on your computer
- Change paths to files and directories as appropriate for use on your computer
- Execute a code block by clicking on the code block and pressing `Shift + Enter`

In [ ]:
# XPRESSpipe-processed data 
file = '/Users/jordan/Desktop/xpressyourself_manuscript/isrib_analysis/isrib_comp_test/isrib_comp_v98_truncated_count_table.tsv'

# Location and filename of original count data 
original = '/Users/jordan/Desktop/xpressyourself_manuscript/isrib_analysis/ingolia_counts_table.txt'

# Define location to save plots to
plot_dir = '/Users/jordan/Desktop/xpressyourself_manuscript/isrib_analysis/plots/'

# Directory where data for DESeq2 should be output
deseq_directory = '/Users/jordan/Desktop/xpressyourself_manuscript/isrib_analysis/isrib_de/xpresspipe_data_deseq2/'

# Define location to save DESeq2 plots to
deseq_plot_directory = '/Users/jordan/Desktop/xpressyourself_manuscript/isrib_analysis/isrib_de/plots/'

# Path and filename for original data reprocessed with DESeq2
dir_og2 = '/Users/jordan/Desktop/xpressyourself_manuscript/isrib_analysis/isrib_de/original_data_deseq2/'

# GTF reference file 
gtf_file = '/Users/jordan/Desktop/reference/Homo_sapiens.GRCh38.98.gtf'

# SRA sample table info
sra_file = '/Users/jordan/Desktop/xpressyourself_manuscript/isrib_analysis/GSE65778_table.txt'

# Location of comparison count tables between methods and with multiple approaches
comparison_tables_directory = '/Users/jordan/Desktop/xpressyourself_manuscript/isrib_analysis/isrib_comp_test/'

### Set global gene lists

In [ ]:
ingolia_tm = [
    'FAM50B','CHST2','LNP1','AX747756','ACRC',
    'ATF5','ANKRD23','ATF4','C19orf48','TEX14',
    'SLC35A4','DDIT3','RAB35','AK293147','PTP4A1',
    'ANKRD36C','NPIP','ANKMY1','UCP2','NPIPL1',
    'PNRC2','C7orf31','PPP1R15A','IFRD1','BCL2L11',
    'OBSCN','SLC24A1','NPIPL1','NPIPL1','ZNF22',
    'NAIP','NPIP','DNAH1','KIAA1377','FMO4',
    'CCDC150','NPIP','AK310228','ZSCAN5A','FAM13B',
    'NPIPL1','ZNF79','HOXA6','NPIPL2','SLMO1',
    'ARMCX4','AK310228','LOC100288332','SAT1','FLJ00285',
    'AK097143','AX748249','AGAP4','FLJ00322','GOLGA8A',
    'GOLGA8B','ZNF432','FSBP','CPS1','MATK',
    'EPB41L4B','HCN2','GLRB','KIAA1841','MYO5B',
    'EHBP1L1','MAPK8IP1','SH3RF3','DET1','PTPN21',
    'VANGL2','MMP15','MAP3K10','PAQR5','GAL',
    'RPP25','FLJ44955','DQ576756']

ingolia_tmisrib = [
    'AK310228','AK310228','NPIP','AK302511','NPIP',
    'LOC100288332','STARD9','PKD1','ANKRD36']

ingolia_isrib = [
    'AK310228','AK310228','AK302511','NPIP']

# Initialize lists of Ingolia highlights
isr = [
    'ATF4',
    'ATF5',
    'PPP1R15A',
    'DDIT3']

uorf_targets = [
    'SLC35A4',
    'PTP4A1',
    'UCP2',
    'C7orf31',
    'BCL2L11',
    'PNRC2',
    'SAT1']

### Define functions needed for processing and plotting

#### Define methods for correlation analysis within and between methods

In [ ]:
"""Collapse technical replicates and rename samples
"""
def name_map(
        data,
        map):

    name_dict = pd.Series(map.ingolia_name.values,index=map.Run).to_dict()
    data_c = data.copy()
    data_c.columns = data_c.columns.to_series().map(name_dict)
    data_sum = data_c.groupby(data_c.columns, axis=1).sum()

    return data_sum

"""Make supplemental figure 4
"""
def make_paralogs_analysis(
    data,
    original_data,
    title):

    # Init plotting space
    fig, axes = plt.subplots(
        nrows = 2,
        ncols = 2,
        figsize = (20, 20),
        subplot_kw = {
            'facecolor':'none'},
        sharey=True,
        sharex=True) # Create shared axis for cleanliness
    plt.subplots_adjust(
        bottom = 0.1)
    plt.yticks([0,1,2,3,4,5]) # Limit axis labels to ints
    plt.xticks([0,1,2,3,4,5])

    x = 0
    file_number = 0
    file_list = [
        'ribo_untr_a',
        'ribo_tm_a',
        'untr_a_hek',
        'tm_a_hek'] # Designate sample order

    for y in range(4):

        # Get data as array-like for samples being compared
        data_c1 = data.copy()
        data_c2 = original_data.copy()

        sample_a = data_c1[file_list[file_number]].values.tolist()
        sample_a = [x + 1 for x in sample_a]
        sample_a = np.array(sample_a).astype(np.float)
        sample_a = np.ndarray.tolist(sample_a)

        sample_b = data_c2[file_list[file_number]].values.tolist()
        sample_b = [x + 1 for x in sample_b]
        sample_b = np.array(sample_b).astype(np.float)
        sample_b = np.ndarray.tolist(sample_b)

        # Determine subplot location
        if file_number in [0,1]:
            ax_x = file_number % 2
            ax_y = 0
        elif file_number in [2,3]:
            ax_x = file_number % 2
            ax_y = 1
        else:
            print('oops')

        # Plot data
        axes[ax_y, ax_x].tick_params(axis='both', labelsize=32)
        axes[ax_y, ax_x].scatter(np.log10(sample_a), np.log10(sample_b), s=5,c='grey')

        genes = ['EIF3C','NOMO2','KCNQ2','RPL39','TUBA1A']
        colors = ['red','blue','green','purple','orange']
        for x in range(5):

            s1 = (data_c1 + 1).at[genes[x], file_list[file_number]]
            s2 = (data_c2 + 1).at[genes[x], file_list[file_number]]

            axes[ax_y, ax_x].scatter(np.log10(s1), np.log10(s2), s=100,c=colors[x])

        axes[ax_y, ax_x].axhline(0, ls='-', color='black', xmin=0.0457, xmax=1) # Create axis lines
        axes[ax_y, ax_x].axvline(0, ls='-', color='black', ymin=0.0457, ymax=1)
        file_number += 1 # Plot counter


    # Create shared row/column titles
    for ax in axes[:,0]:
        ax.set_ylabel('log$_1$$_0$(counts)', fontsize=32)

    cols = ['Untreated','Tm']
    for ax, col in zip(axes[0], cols):
        ax.set_xlabel(col, fontsize=40)
        ax.xaxis.set_label_position('top')

    rows = ['Ribo-seq','RNA-seq']
    for ax, rows in zip(axes[:,1], rows):
        ax.set_ylabel(rows, fontsize=40, labelpad=40, rotation=270)
        ax.yaxis.set_label_position('right')

    for ax in axes[1]:
        ax.set_xlabel('log$_1$$_0$(counts)', fontsize=32)
        ax.xaxis.set_label_position('bottom')

    fig.savefig(
        str(plot_dir) + str(title),
        dpi = 600,
        bbox_inches = 'tight')

# Make figure 2B
def make_external_correlations(
        data,
        original_data,
        title):

    fig, axes = plt.subplots(
        nrows = 2,
        ncols = 2,
        figsize = (20, 20),
        subplot_kw = {
            'facecolor':'none'},
        sharey=True) # Create shared axis for cleanliness
    plt.subplots_adjust(
        bottom = 0.1)
    plt.yticks([0,1,2,3,4,5]) # Limit axis labels to ints
    plt.xticks([0,1,2,3,4,5])
    plt.axis('equal')

    x = 0
    file_number = 0
    file_list = [
        'ribo_tm_a',
        'isrib_b_hek'] # Designate sample order

    for y in range(2):

        # Get data as array-like for samples being compared
        data_c1 = data.copy()
        data_c2 = original_data.copy()

        sample_a = data_c1[file_list[file_number]].values.tolist()
        sample_a = [x + 1 for x in sample_a]
        sample_a = np.array(sample_a).astype(np.float)
        sample_a = np.ndarray.tolist(sample_a)

        sample_b = data_c2[file_list[file_number]].values.tolist()
        sample_b = [x + 1 for x in sample_b]
        sample_b = np.array(sample_b).astype(np.float)
        sample_b = np.ndarray.tolist(sample_b)

        # Run Spearman R linreg for non-normal data
        rho, p_value = stats.spearmanr(sample_a, sample_b)

        # Determine subplot location
        if file_number in [0,1]:
            ax_x = file_number % 2
            ax_y = 0
        else:
            print('oops')

        # Format p value
        if p_value < 0.001:
            p_val = '< 0.001'
        else:
            p_val = '= {:.3f}'.format(round(p_value, 3))

        rho = '{:.3f}'.format(round(rho, 3))

        # Plot data
        axes[ax_y, ax_x].tick_params(axis='both', labelsize=32)
        axes[ax_y, ax_x].scatter(np.log10(sample_a), np.log10(sample_b), s=1,c='black')
        axes[ax_y, ax_x].set_title(r"$\rho$" + ' = ' + str(rho) + '\nP ' + p_val, y=0.1, x=0.9, fontsize=32) # Format titles
        axes[ax_y, ax_x].axhline(0, ls='-', color='black', xmin=0.0457, xmax=1) # Create axis lines
        axes[ax_y, ax_x].axvline(0, ls='-', color='black', ymin=0.0457, ymax=1)
        file_number += 1 # Plot counter
        x += 2
        print(rho)

    # Create shared row/column titles
    for ax in axes[:,0]:
        ax.set_ylabel('log$_1$$_0$(counts)', fontsize=32)

    cols = ['RPF Tm RepA','mRNA ISRIB RepB']
    for ax, col in zip(axes[0], cols):
        ax.set_xlabel(col, fontsize=40)
        ax.xaxis.set_label_position('top')
    cols = ['log$_1$$_0$(counts)','log$_1$$_0$(counts)']
    for ax, col in zip(axes[1], cols):
        ax.set_xlabel(col, fontsize=32, labelpad=30)
        ax.xaxis.set_label_position('top')

    fig.savefig(
        str(plot_dir) + str(title),
        dpi = 600,
        bbox_inches = 'tight')

def make_external_correlations_supplement(
        data,
        original_data,
        title,
        interactive=False):

    fig, axes = plt.subplots(
        nrows = 4,
        ncols = 4,
        figsize = (20, 20),
        subplot_kw = {
            'facecolor':'none'},
        sharex=True, sharey=True) # Create shared axis for cleanliness
    plt.subplots_adjust(
        bottom = 0.1)
    plt.yticks([0,1,2,3,4,5]) # Limit axis labels to ints
    plt.xticks([0,1,2,3,4,5])

    file_number = 0
    file_list = [
        'ribo_untr_a',
        'ribo_untr_b',
        'untr_a_hek',
        'untr_b_hek',
        'ribo_tm_a',
        'ribo_tm_b',
        'tm_a_hek',
        'tm_b_hek',
        'ribo_tmisrib_a',
        'ribo_tmisrib_b',
        'tmisrib_a_hek',
        'tmisrib_b_hek',
        'ribo_isrib_a',
        'ribo_isrib_b',
        'isrib_a_hek',
        'isrib_b_hek'] # Designate sample order

    for x in file_list:

        if interactive == True:
            merged_best = pd.concat([data[[str(x)]], original_data[[str(x)]]], axis=1, sort=False)
            merged_best.columns = ['xpresspipe', 'original']
            merged_best['genes'] = merged_best.index
            sc = px.scatter(
                merged_best,
                x='xpresspipe',
                y='original',
                hover_name='genes',
                log_x=True,
                log_y=True,
                opacity=0.4,
                width=1400,
                height=1000,
                title=str(x))

            py.offline.plot(
                sc, 
                filename=str(plot_dir) + 'interactive_' + str(x) + '.html')

        # Get data as array-like for samples being compared
        data_c1 = data.copy()
        data_c2 = original_data.copy()

        sample_a = data_c1[x].values.tolist()
        sample_a = [x + 1 for x in sample_a]
        sample_a = np.array(sample_a).astype(np.float)
        sample_a = np.ndarray.tolist(sample_a)

        sample_b = data_c2[x].values.tolist()
        sample_b = [x + 1 for x in sample_b]
        sample_b = np.array(sample_b).astype(np.float)
        sample_b = np.ndarray.tolist(sample_b)

        # Run Spearman R linreg for non-normal data
        rho, p_value = stats.spearmanr(sample_a, sample_b)

        # Determine subplot location
        if file_number in [0,1,2,3]:
            ax_x = file_number % 4
            ax_y = 0
        elif file_number in [4,5,6,7]:
            ax_x = file_number - 4
            ax_y = 1
        elif file_number in [8,9,10,11]:
            ax_x = file_number - 8
            ax_y = 2
        elif file_number in [12,13,14,15]:
            ax_x = file_number - 12
            ax_y = 3
        else:
            print('oops')

        # Format p value
        if p_value < 0.001:
            p_val = '< 0.001'
        else:
            p_val = '= {:.3f}'.format(round(p_value, 3))

        rho = '{:.3f}'.format(round(rho, 3))

        # Plot data
        axes[ax_y, ax_x].tick_params(axis='both', labelsize=32)
        axes[ax_y, ax_x].scatter(np.log10(sample_a), np.log10(sample_b), s=1,c='black')
        axes[ax_y, ax_x].set_title(r"$\rho$" + ' = ' + str(rho) + '\nP ' + p_val, y=0.1, x=0.8, fontsize=20) # Format titles
        axes[ax_y, ax_x].axhline(0, ls='-', color='black', xmin=0.05, xmax=1) # Create axis lines
        axes[ax_y, ax_x].axvline(0, ls='-', color='black', ymin=0.05, ymax=0.88)
        file_number += 1 # Plot counter
        print(rho)

    # Create shared row/column titles
    count_label = ['log$_1$$_0$(counts)','log$_1$$_0$(counts)','log$_1$$_0$(counts)','log$_1$$_0$(counts)']

    cols = ['RPF RepA','RPF RepB','mRNA RepA','mRNA RepB']
    for ax, col in zip(axes[0], cols):
        ax.set_xlabel(col, fontsize=40)
        ax.xaxis.set_label_position('top')

    for ax in axes[3]:
        ax.set_xlabel('log$_1$$_0$(counts)', fontsize=32)
        ax.xaxis.set_label_position('bottom')

    for ax in axes[:,0]:
        ax.set_ylabel('log$_1$$_0$(counts)', fontsize=32)

    rows = ['Untreated','Tm','Tm + ISRIB','ISRIB']
    for ax, rows in zip(axes[:,3], rows):
        ax.set_ylabel(rows, fontsize=40, labelpad=40, rotation=270)
        ax.yaxis.set_label_position('right')

    fig.savefig(
        str(plot_dir) + str(title),
        dpi = 600,
        bbox_inches = 'tight')


def make_spearman_pearson_correlations(
        data,
        original_data,
        title):

    data_t = np.log10(data + 1)
    original_data_t = np.log10(original_data + 1)

    file_list = [
        'ribo_untr_a',
        'ribo_untr_b',
        'untr_a_hek',
        'untr_b_hek',
        'ribo_tm_a',
        'ribo_tm_b',
        'tm_a_hek',
        'tm_b_hek',
        'ribo_tmisrib_a',
        'ribo_tmisrib_b',
        'tmisrib_a_hek',
        'tmisrib_b_hek',
        'ribo_isrib_a',
        'ribo_isrib_b',
        'isrib_a_hek',
        'isrib_b_hek'] # Designate sample order

    # Run Spearman on raw counts
    spear_list = []
    for x in file_list:

        # Get data as array-like for samples being compared
        data_c1 = data.copy()
        data_c2 = original_data.copy()

        sample_a = data_c1[x].values.tolist()
        sample_a = [x + 1 for x in sample_a]
        sample_a = np.array(sample_a).astype(np.float)
        sample_a = np.ndarray.tolist(sample_a)

        sample_b = data_c2[x].values.tolist()
        sample_b = [x + 1 for x in sample_b]
        sample_b = np.array(sample_b).astype(np.float)
        sample_b = np.ndarray.tolist(sample_b)

        # Run Spearman R linreg for non-normal data
        rho, spear_p_value = stats.spearmanr(sample_a, sample_b)
        spear_list.append(rho)

    # Run Pearson on log transformed data
    pear_list = []
    for x in file_list:

        # Get data as array-like for samples being compared
        data_c1 = data_t.copy()
        data_c2 = original_data_t.copy()

        sample_a = data_c1[x].values.tolist()
        sample_a = [x + 1 for x in sample_a]
        sample_a = np.array(sample_a).astype(np.float)
        sample_a = np.ndarray.tolist(sample_a)

        sample_b = data_c2[x].values.tolist()
        sample_b = [x + 1 for x in sample_b]
        sample_b = np.array(sample_b).astype(np.float)
        sample_b = np.ndarray.tolist(sample_b)

        r, pear_p_value = stats.pearsonr(sample_a, sample_b)
        pear_list.append(r)

    spear_list_reps = []
    file_number = 0
    for x in range(int(len(file_list)/2)):

        # Get data as array-like for samples being compared
        data_c1 = data.copy()

        sample_a = data_c1[file_list[file_number]].values.tolist()
        sample_a = [x + 1 for x in sample_a]
        sample_a = np.array(sample_a).astype(np.float)
        sample_a = np.ndarray.tolist(sample_a)

        sample_b = data_c1[file_list[file_number+1]].values.tolist()
        sample_b = [x + 1 for x in sample_b]
        sample_b = np.array(sample_b).astype(np.float)
        sample_b = np.ndarray.tolist(sample_b)

        file_number += 2

        # Run Spearman R linreg for non-normal data
        rho, spear_p_value = stats.spearmanr(sample_a, sample_b)
        spear_list_reps.append(rho)


    pear_list_reps = []
    file_number = 0
    for x in range(int(len(file_list)/2)):

        # Get data as array-like for samples being compared
        data_c1 = data_t.copy()

        sample_a = data_c1[file_list[file_number]].values.tolist()
        sample_a = [x + 1 for x in sample_a]
        sample_a = np.array(sample_a).astype(np.float)
        sample_a = np.ndarray.tolist(sample_a)

        sample_b = data_c1[file_list[file_number+1]].values.tolist()
        sample_b = [x + 1 for x in sample_b]
        sample_b = np.array(sample_b).astype(np.float)
        sample_b = np.ndarray.tolist(sample_b)

        file_number += 2

        r, pear_p_value = stats.pearsonr(sample_a, sample_b)
        pear_list_reps.append(r)

    data = [
        spear_list_reps,
        pear_list_reps,
        spear_list,
        pear_list]

    names = [
        'Spearman Replicates',
        'Pearson Replicates',
        'Spearman Method Comparison',
        'Pearson Method Comparison']

    counter = 0
    df = pd.DataFrame(columns = ['Value', 'Type'])

    for x in range(4):
        l = data[x]
        n = names[x]

        for y in l:
            df.loc[counter] = [float(y), str(n)]
            counter += 1

    df1 = df.loc[df['Type'].str.contains('Replicates')]
    df1['Type'] = df1['Type'].str.split(' ').str[0]
    df1.columns = ['Coefficient Value', 'Replicates']
    ax1 = sns.violinplot(x="Coefficient Value", y="Replicates", data=df1)
    plt.savefig(
        str(plot_dir) + 'replicates_' + str(title),
        dpi = 1800,
        bbox_inches = 'tight')
    plt.close()

    df2 = df.loc[df['Type'].str.contains('Method')]
    df2['Type'] = df2['Type'].str.split(' ').str[0]
    df2.columns = ['Coefficient Value', 'Method Comparison']
    ax2 = sns.violinplot(x="Coefficient Value", y="Method Comparison", data=df2)
    plt.savefig(
        str(plot_dir) + 'method_' + str(title),
        dpi = 1800,
        bbox_inches = 'tight')
    plt.close()

"""Make Figure 2A
"""
def make_internal_correlations(
        data,
        title):

    fig, axes = plt.subplots(
        nrows = 2,
        ncols = 2,
        figsize = (20, 20),
        subplot_kw = {
            'facecolor':'none'}) # Create shared axis for cleanliness
    plt.subplots_adjust(
        bottom = 0.1)
    plt.yticks([0,1,2,3,4,5]) # Limit axis labels to ints
    plt.xticks([0,1,2,3,4,5])

    x = 0
    file_number = 0
    file_list = [
        'ribo_tm_a',
        'ribo_tm_b',
        'isrib_a_hek',
        'isrib_b_hek'] # Designate sample order

    for y in range(int(len(file_list)/2)):

        # Get data as array-like for samples being compared
        data_c1 = data.copy()

        sample_a = data_c1[file_list[x]].values.tolist()
        sample_a = [x + 1 for x in sample_a]
        sample_a = np.array(sample_a).astype(np.float)
        sample_a = np.ndarray.tolist(sample_a)

        sample_b = data_c1[file_list[x+1]].values.tolist()
        sample_b = [x + 1 for x in sample_b]
        sample_b = np.array(sample_b).astype(np.float)
        sample_b = np.ndarray.tolist(sample_b)

        # Run Spearman R linreg for non-normal data
        rho, p_value = stats.spearmanr(sample_a, sample_b)

        # Determine subplot location
        if file_number in [0,1]:
            ax_x = file_number % 2
            ax_y = 0
        else:
            print('oops')

        # Format p value
        if p_value < 0.001:
            p_val = '< 0.001'
        else:
            p_val = '= {:.3f}'.format(round(p_value, 3))

        rho = '{:.3f}'.format(round(rho, 3))

        # Plot data
        axes[ax_y, ax_x].tick_params(axis='both', labelsize=32)
        axes[ax_y, ax_x].scatter(np.log10(sample_a), np.log10(sample_b), s=1,c='black')
        axes[ax_y, ax_x].set_title(r"$\rho$" + ' = ' + str(rho) + '\nP ' + p_val, y=0.1, x=0.9, fontsize=32) # Format titles
        axes[ax_y, ax_x].axhline(0, ls='-', color='black', xmin=0.0457, xmax=1) # Create axis lines
        axes[ax_y, ax_x].axvline(0, ls='-', color='black', ymin=0.0457, ymax=1)
        file_number += 1 # Plot counter
        x += 2
        print(rho)

    # Create shared row/column titles

    for ax in axes[:,0]:
        ax.set_ylabel('log$_1$$_0$(counts)', fontsize=32)

    cols = ['RPF Tm','mRNA ISRIB']
    for ax, col in zip(axes[0], cols):
        ax.set_xlabel(col, fontsize=40)
        ax.xaxis.set_label_position('top')
    cols = ['log$_1$$_0$(counts)','log$_1$$_0$(counts)']
    for ax, col in zip(axes[1], cols):
        ax.set_xlabel(col, fontsize=32, labelpad=30)
        ax.xaxis.set_label_position('top')

    fig.savefig(
        str(plot_dir) + str(title),
        dpi = 600,
        bbox_inches = 'tight')

def make_internal_correlations_supplement(
        data,
        title):

    fig, axes = plt.subplots(
        nrows = 2,
        ncols = 4,
        figsize = (20, 10),
        subplot_kw = {
            'facecolor':'none'},
        sharex=True, sharey=True) # Create shared axis for cleanliness
    plt.subplots_adjust(
        bottom = 0.1)
    plt.yticks([0,1,2,3,4,5]) # Limit axis labels to ints
    plt.xticks([0,1,2,3,4,5])

    x = 0
    file_number = 0
    file_list = [
        'ribo_untr_a',
        'ribo_untr_b',
        'untr_a_hek',
        'untr_b_hek',
        'ribo_tm_a',
        'ribo_tm_b',
        'tm_a_hek',
        'tm_b_hek',
        'ribo_tmisrib_a',
        'ribo_tmisrib_b',
        'tmisrib_a_hek',
        'tmisrib_b_hek',
        'ribo_isrib_a',
        'ribo_isrib_b',
        'isrib_a_hek',
        'isrib_b_hek'] # Designate sample order

    for y in range(int(len(file_list)/2)):

        # Get data as array-like for samples being compared
        data_c1 = data.copy()

        sample_a = data_c1[file_list[x]].values.tolist()
        sample_a = [x + 1 for x in sample_a]
        sample_a = np.array(sample_a).astype(np.float)
        sample_a = np.ndarray.tolist(sample_a)

        sample_b = data_c1[file_list[x+1]].values.tolist()
        sample_b = [x + 1 for x in sample_b]
        sample_b = np.array(sample_b).astype(np.float)
        sample_b = np.ndarray.tolist(sample_b)

        # Run Spearman R linreg for non-normal data
        rho, p_value = stats.spearmanr(sample_a, sample_b)

        # Determine subplot location
        if file_number in [0,1,2,3]:
            ax_x = file_number % 4
            ax_y = 0
        elif file_number in [4,5,6,7]:
            ax_x = file_number - 4
            ax_y = 1
        else:
            print('oops')

        # Format p value
        if p_value < 0.001:
            p_val = '< 0.001'
        else:
            p_val = '= {:.3f}'.format(round(p_value, 3))

        rho = '{:.3f}'.format(round(rho, 3))

        # Plot data
        axes[ax_y, ax_x].tick_params(axis='both', labelsize=32)
        axes[ax_y, ax_x].scatter(np.log10(sample_a), np.log10(sample_b), s=1,c='black')
        axes[ax_y, ax_x].set_title(r"$\rho$" + ' = ' + str(rho) + '\nP ' + p_val, y=0.1, x=0.8, fontsize=20) # Format titles
        axes[ax_y, ax_x].axhline(0, ls='-', color='black', xmin=0.05, xmax=1) # Create axis lines
        axes[ax_y, ax_x].axvline(0, ls='-', color='black', ymin=0.05, ymax=1)
        file_number += 1 # Plot counter
        x += 2
        print(rho)

    # Create shared row/column titles

    for ax in axes[:,0]:
        ax.set_ylabel('log$_1$$_0$(counts)', fontsize=32)

    cols = ['RPF Untreated','mRNA Untreated','RPF Tm','mRNA Tm']
    for ax, col in zip(axes[0], cols):
        ax.set_xlabel(col, fontsize=32)
        ax.xaxis.set_label_position('top')

    cols = ['RPF Tm + ISRIB','mRNA Tm + ISRIB','RPF ISRIB','mRNA ISRIB']
    for ax, col in zip(axes[1], cols):
        ax.set_xlabel(col, fontsize=32)
        ax.xaxis.set_label_position('top')

    fig.savefig(
        str(plot_dir) + str(title),
        dpi = 600,
        bbox_inches = 'tight')

# Read in gene dictionary
def make_ref(
        ref_file,
        gene_list):

    df = pd.read_csv(
        str(ref_file) ,
        sep='\t',
        index_col=2)
    del df.index.name
    df = df[['NCBI gene ID', 'UniProt accession']]
    df['NCBI gene ID'] = df['NCBI gene ID'].astype(pd.Int32Dtype())

    df['index'] = df.index
    df_keep = df.drop_duplicates(subset='index', keep='first')
    df_genes = df_keep.reindex(gene_list)
    df_genes = df_genes.drop('index', axis=1)

    return df_genes

"""Read in single-pass XPRESSpipe processed read data quantified with HTSeq using non-de-deuplicated alignments
"""
def get_data(
        file,
        sample_suffix='_1_Aligned'):

    data = pd.read_csv(
        file,
        sep = '\t',
        index_col = 0)

    data.index = data.index.str.split('.').str[0]

    # Accessed via:
    # $ curl -O ftp://ftp.ensembl.org/pub/release-96/gtf/homo_sapiens/Homo_sapiens.GRCh38.96.gtf.gz
    # $ gzip -d Homo_sapiens.GRCh38.96.gtf.gz
    data = convert_names(
        data,
        str(gtf_file))
    data.columns = data.columns.str.replace(str(sample_suffix), '')
    data.shape

    # Combine lanes
    sra_info = pd.read_csv(
        str(sra_file),
        sep = '\t')
    data = name_map(data, sra_info)
    data = data.groupby(level=0).sum() # Combine duplicate named genes

    return data

"""Run correlations between sample alignments
"""
def cycle_external_correlations_supplement(
        file,
        original):

    try:
        data = get_data(file, sample_suffix='_1_Aligned')
    except:
        data = get_data(file, sample_suffix='__Aligned')

    data_threshold = data.loc[data[['untr_a_hek', 'untr_b_hek', 'tm_a_hek', 'tm_b_hek', 'tmisrib_a_hek', 'tmisrib_b_hek', 'isrib_a_hek', 'isrib_b_hek']].min(axis=1) >= 10] # Apply threshold to data
    data_rpm = rpm(data_threshold)
    data_genes = data_rpm.index.tolist()

    original_rpm = rpm(original)
    original_genes = original_rpm.index.tolist()

    common_genes = list(set(data_genes).intersection(original_genes))

    data_common = data_rpm.reindex(index = common_genes)
    original_common = original_rpm.reindex(index = common_genes)

    make_external_correlations_supplement(
        data_common,
        original_common,
        str(file.split('/')[-1][:-4]) + '_external_correlations_summary_htseq_all.png')

#### Differential Expression Methods

In [ ]:
"""Prep data vectors
"""
def prep_data(
        data,
        y_name,
        x_name):

    ribo_values = data[[y_name]].values.tolist()
    ribo_values = np.array(ribo_values).astype(np.float)
    ribo_values = np.ndarray.tolist(ribo_values)

    rna_values = data[[x_name]].values.tolist()
    rna_values = np.array(rna_values).astype(np.float)
    rna_values = np.ndarray.tolist(rna_values)

    return ribo_values, rna_values

"""Plot optional data
"""
def plot_optional(
        ax,
        ribo_values,
        rna_values,
        data,
        data_subset,
        color,
        alpha=1,
        regulation_list=None,
        significance_list=None,
        label=False,
        optional_genes_1=[],
        optional_genes_2=[]):

    # Plot first pass data
    ax.scatter(
        rna_values,
        ribo_values,
        s=80,
        c=color,
        alpha=alpha)

    # Plot significance coloring
    for index, row in data.iterrows():
        if index in regulation_list:
            if index in significance_list:
                ax.scatter(
                    row[1],
                    row[0],
                    s=20,
                    c='black',
                    alpha=1)

            else:
                ax.scatter(
                    row[1],
                    row[0],
                    s=20,
                    c='gray',
                    alpha=1)

        else:
            pass

    # Add labels
    if label == True:

        for index, row in data_subset.iterrows():

            if index in optional_genes_1 + optional_genes_2:

                for x in optional_genes_1:

                    if index == x:
                        ax.text(
                            row[1] + 0.1,
                            row[0] - 0.07,
                            str(index),
                            horizontalalignment='left',
                            size='medium',
                            color=color,
                            weight='semibold')

                for y in optional_genes_2:

                    if index == y:
                        ax.text(
                            row[1] - 0.05,
                            row[0] - 0.2,
                            str(index),
                            horizontalalignment='right',
                            size='medium',
                            color=color,
                            weight='semibold')

            else:
                ax.text(
                    row[1] - 0.1,
                    row[0] - 0.07,
                    str(index),
                    horizontalalignment='right',
                    size='medium',
                    color=color,
                    weight='semibold')

    return ax

"""Plot ribo-seq data on X/Y figure
"""
def rp_plot(
        data,
        y_fc,
        x_fc,
        y_p,
        x_p,
        y_name,
        x_name,
        te_data,
        te_p,
        title,
        list_up,
        list_down,
        list_down_custom,
        genes=[],
        label_up=True,
        label_down=True,
        label_down_custom=True):

    # Get relevant data
    data_fig = data[[y_fc, x_fc, y_p, x_p]].copy()

    # Remove infinite and missing values
    data_fig[[y_fc, x_fc]] = data_fig[[y_fc, x_fc]].replace([np.inf, -np.inf], np.nan)
    data_fig = data_fig.dropna(subset=[y_fc, x_fc])


    # Initialize plotting space
    fig, ax = plt.subplots(1,1, figsize=(7,7))
    plt.grid(False)
    ax.axhline(0, ls='-', color='black') # Create central axis line
    ax.axvline(0, ls='-', color='black')
    rect = patches.Rectangle((-1,-1),2,2,linewidth=1.5,edgecolor='gray',facecolor='none') # Create significance zone
    ax.add_patch(rect)
    ax.set_ylabel('Ribo-Seq log$_2$(' + str(y_name) + '/' + str(x_name) + ')', fontsize=16) # Set axis labels
    ax.set_xlabel('mRNA-Seq log$_2$(' + str(y_name) + '/' + str(x_name) + ')', fontsize=16)
    ax.set_xlim(-3.5,3.5) # Set axis limits
    ax.set_ylim(-3.5,3.5)
    x = [-3,-2,-1,0,1,2,3] # Set axis spacing
    ticks = ["-3","-2","-1","0","1","2","3"]
    ax.set_facecolor("#FFFFFF") # Set background color

    # Set X and Y axis scales
    ax.set_xticks(x)
    ax.set_xticklabels(ticks, fontsize=16)
    ax.set_xticklabels([""]*len(x), minor=True)

    ax.set_yticks(x)
    ax.set_yticklabels(ticks, fontsize=16)
    ax.set_yticklabels([""]*len(x), minor=True)

    # Prep data for plotting
    ribo_all, rna_all = prep_data(
        data=data_fig,
        y_name=y_fc,
        x_name=x_fc)

    # Prep significant hits for plotting
    sig_list = te_data.loc[te_data[te_p] < 0.1].index.tolist()
    data_sig = data_fig.reindex(sig_list)
    ribo_sig, rna_sig = prep_data(
        data=data_sig,
        y_name=y_fc,
        x_name=x_fc)

    # Prep option gene sets
    if list_up != None:
        data_fig_up = data_fig.reindex(list_up)
        ribo_up, rna_up = prep_data(
            data=data_fig_up,
            y_name=y_fc,
            x_name=x_fc)

    if list_down != None:
        data_fig_down = data_fig.reindex(list_down)
        ribo_down, rna_down = prep_data(
            data=data_fig_down,
            y_name=y_fc,
            x_name=x_fc)

    if list_down_custom != None:
        data_fig_down_custom = data_fig.reindex(list_down_custom)
        ribo_down_custom, rna_down_custom = prep_data(
            data=data_fig_down_custom,
            y_name=y_fc,
            x_name=x_fc)

    #Plot data
    ax.scatter(rna_all, ribo_all, s=2.5,c='gray',alpha=0.5)
    ax.scatter(rna_sig, ribo_sig, s=5,c='black',alpha=1)

    if list_down != None:

        ax = plot_optional(
            ax=ax,
            ribo_values=ribo_down,
            rna_values=rna_down,
            data=data_fig,
            data_subset=data_fig_down,
            color='#1b9e77',
            regulation_list=list_down,
            significance_list=sig_list,
            label=label_down,
            optional_genes_1=[
                'PNRC2',
                'SAT1']
            )

    if list_down_custom != None:

        ax = plot_optional(
            ax=ax,
            ribo_values=ribo_down_custom,
            rna_values=rna_down_custom,
            data=data_fig,
            data_subset=data_fig_down_custom,
            color='#d95f02',
            regulation_list=list_down_custom,
            significance_list=sig_list,
            label=label_down_custom,
            optional_genes_1=[
                'RPS15A',
                'HIST2H3D',
                'RPL27'],
            optional_genes_2=[
                'HSPA8']
            )

    if list_up != None:

        ax = plot_optional(
            ax=ax,
            ribo_values=ribo_up,
            rna_values=rna_up,
            data=data_fig,
            data_subset=data_fig_up,
            color='#7570b3',
            regulation_list=list_up,
            significance_list=sig_list,
            label=label_up,
            optional_genes_1=[
                'ATF4',
                'DDIT3',
                'PPP1R15A']
            )

    plt.savefig(
        str(deseq_plot_directory) + str(title),
        dpi=3600,
        bbox_inches='tight')

    plt.close()

"""Import DE data
"""
def parse_de_data(
        directory,
        check_file):

    check_data = pd.read_csv(
        str(directory) + str(check_file),
        sep='\t',
        index_col=0)
    check_list = check_data.index.tolist()

    # Import DESeq2 TE data
    tm_data = pd.read_csv(str(directory) + 'tm_counts_diffx.tsv', sep='\t')
    tmisrib_data = pd.read_csv(str(directory) + 'tmisrib_counts_diffx.tsv', sep='\t')
    isrib_data = pd.read_csv(str(directory) + 'isrib_counts_diffx.tsv', sep='\t')

    ### Clean up this data
    tm_data = tm_data.drop(['baseMean','lfcSE','stat'],axis=1)
    tm_data.columns = ['tm_log2FC','tm_p','tm_padj']

    tmisrib_data = tmisrib_data.drop(['baseMean','lfcSE','stat'],axis=1)
    tmisrib_data.columns = ['tmisrib_log2FC','tmisrib_p','tmisrib_padj']

    isrib_data = isrib_data.drop(['baseMean','lfcSE','stat'],axis=1)
    isrib_data.columns = ['isrib_log2FC','isrib_p','isrib_padj']

    merged_data = pd.concat([tm_data, tmisrib_data, isrib_data], axis=1, sort=False)

    # Import DESeq2 RNA and RPF data
    tm_ribo_data = pd.read_csv(str(directory) + 'tm_ribo_counts_diffx.tsv', sep='\t')
    tm_rna_data = pd.read_csv(str(directory) + 'tm_rna_counts_diffx.tsv', sep='\t')

    tmisrib_ribo_data = pd.read_csv(str(directory) + 'tmisrib_ribo_counts_diffx.tsv', sep='\t')
    tmisrib_rna_data = pd.read_csv(str(directory) + 'tmisrib_rna_counts_diffx.tsv', sep='\t')

    isrib_ribo_data = pd.read_csv(str(directory) + 'isrib_ribo_counts_diffx.tsv', sep='\t')
    isrib_rna_data = pd.read_csv(str(directory) + 'isrib_rna_counts_diffx.tsv', sep='\t')

    ### Clean up this data
    tm_ribo_data = tm_ribo_data.drop(['baseMean','lfcSE','stat','pvalue'],axis=1)
    tm_ribo_data.columns = ['tm_ribo_log2FC','tm_ribo_padj']
    tm_rna_data = tm_rna_data.drop(['baseMean','lfcSE','stat','pvalue'],axis=1)
    tm_rna_data.columns = ['tm_rna_log2FC','tm_rna_padj']

    tmisrib_ribo_data = tmisrib_ribo_data.drop(['baseMean','lfcSE','stat','pvalue'],axis=1)
    tmisrib_ribo_data.columns = ['tmisrib_ribo_log2FC','tmisrib_ribo_padj']
    tmisrib_rna_data = tmisrib_rna_data.drop(['baseMean','lfcSE','stat','pvalue'],axis=1)
    tmisrib_rna_data.columns = ['tmisrib_rna_log2FC','tmisrib_rna_padj']

    isrib_ribo_data = isrib_ribo_data.drop(['baseMean','lfcSE','stat','pvalue'],axis=1)
    isrib_ribo_data.columns = ['isrib_ribo_log2FC','isrib_ribo_padj']
    isrib_rna_data = isrib_rna_data.drop(['baseMean','lfcSE','stat','pvalue'],axis=1)
    isrib_rna_data.columns = ['isrib_rna_log2FC','isrib_rna_padj']

    merged_data_split = pd.concat([tm_ribo_data, tm_rna_data, tmisrib_ribo_data, tmisrib_rna_data, isrib_ribo_data, isrib_rna_data], axis=1, sort=False)

    return merged_data, merged_data_split, check_data

"""TE sanity check
"""
def te_sanity(
        data,
        check_data,
        gene_name):

    check_data_rpm = rpm(check_data)

    print(gene_name)

    print('DESeq2 estimate: ', data.loc[gene_name]['tm_log2FC'])

    # Get gene info for all samples
    gene = check_data_rpm.loc[gene_name][['ribo_tm_a', 'ribo_tm_b', 'ribo_untr_a', 'ribo_untr_b','tm_a_hek','tm_b_hek','untr_a_hek','untr_b_hek']]

    # Calculate Tm TE by summing replicates and calculating RPF / RNA
    gene['tm_te'] = (gene['ribo_tm_a'] + gene['ribo_tm_b']) / (gene['tm_a_hek'] + gene['tm_b_hek'])
    gene['untr_te'] = (gene['ribo_untr_a'] + gene['ribo_untr_b']) / (gene['untr_a_hek'] + gene['untr_b_hek'])

    # Calculate FC of TEs for Treatment / not-treated and take the log2 value of that outpu
    gene['fold_change'] = np.log2(gene['tm_te'] / gene['untr_te'])
    print('Sanity estimate: ', gene['fold_change'])

"""Plot TE for genes of interest
"""
def te_plot(
        data,
        list_up,
        list_down,
        directory,
        plot_all=False,
        dpi=3600):

    # Init plotting space
    fig, ax = plt.subplots()
    plt.yticks([-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10])
    ax.tick_params(axis='x', which='minor',length=0)

    ax.set_facecolor('white')
    ax.grid(color='grey', axis='y')

    # Plot all genes' TE
    if plot_all == True:
        data_de_plot.T.plot.line(
            legend=False,
            color='lightgrey',
            linewidth=0.5,
            ax=ax)

    # Set grid lines
    ax.axvline(0.01, ls='-', color='black')
    ax.axvline(1, ls='-', color='black')
    ax.axvline(2, ls='-', color='black')
    ax.axvline(2.99, ls='-', color='black')
    ax.axhline(0, ls='-', color='black')
    ax.axhline(1, ls='--', color='black')
    ax.axhline(-1, ls='--', color='black')

    # Plot genes of interest
    data_de_plot.loc[list_up].T.plot.line(
        legend=False,
        color='#7570b3',
        linewidth=1,
        ax=ax)
    data_de_plot.loc[list_down].T.plot.line(
        legend=False,
        color='#d95f02',
        linewidth=1,
        ax=ax)
    ax.set_ylabel(u'log$_2$(ΔTE)')

    # Add legend
    from matplotlib.lines import Line2D
    legend_elements = [Line2D([0], [0], color='gray', lw=2, label='All'),
                       Line2D([0], [0], color='#7570b3', lw=2, label='ISR'),
                       Line2D([0], [0], color='#d95f02', lw=2, label='Other')]
    ax.legend(handles=legend_elements, loc='upper right')

    plt.savefig(
        str(directory) + 'te_analysis_down_strict.png',
        dpi=dpi,
        bbox_inches='tight')

    plt.close()

"""Plot Venn diagrams
"""
def venn_plot(
        directory,
        figure_title,
        output_name,
        gene_list_A,
        gene_list_B,
        label_A,
        label_B,
        color_A='#d95f02',
        color_B='#7570b3',
        dpi=3600):

    plt.title(
        figure_title,
        fontsize = 28)

    c = venn2(
        [set(gene_list_A), set(gene_list_B)],
        set_labels = (label_A, label_B))
    c.get_patch_by_id('10').set_color(color_A)
    c.get_patch_by_id('01').set_color(color_B)
    c.get_patch_by_id('10').set_edgecolor('black')
    c.get_patch_by_id('01').set_edgecolor('black')

    for text in c.set_labels:

        text.set_fontsize(22)

    try:
        for text in c.subset_labels:

            text.set_fontsize(18)

    except:
        pass

    plt.savefig(
        directory + output_name,
        dpi=dpi,
        bbox_inches='tight')

    plt.close()

### Process Data

#### Read in XPRESSpipe-processed data and process

In [ ]:
# Input file is protein coding only and truncated, not parsed for longest transcript only
data = get_data(file, sample_suffix='_1_Aligned')

# Clean up data
data_threshold_deseq = data.loc[
    data[[
        'untr_a_hek', 
        'untr_b_hek', 
        'tm_a_hek', 
        'tm_b_hek', 
        'tmisrib_a_hek', 
        'tmisrib_b_hek', 
        'isrib_a_hek', 
        'isrib_b_hek']
    ].min(axis=1) >= 25] # Apply threshold to data

data_threshold = data.loc[
    data[[
        'untr_a_hek', 
        'untr_b_hek', 
        'tm_a_hek', 
        'tm_b_hek', 
        'tmisrib_a_hek', 
        'tmisrib_b_hek', 
        'isrib_a_hek', 
        'isrib_b_hek']
    ].min(axis=1) >= 10] # Apply threshold to data

data_rpm = rpm(data_threshold)

#### Export processed XPRESSpipe data for DESeq2

In [ ]:
# Export truth set for downsteam checks
data_threshold_deseq.to_csv(
    deseq_directory + 'ISRIBxpresspipe_thresholded_counts.tsv',
    sep='\t')

# Define sample sets
untr_mrna = ['untr_a_hek', 'untr_b_hek']
untr_ribo = ['ribo_untr_a', 'ribo_untr_b']
tm_mrna = ['tm_a_hek', 'tm_b_hek']
tm_ribo = ['ribo_tm_a', 'ribo_tm_b']
tmisrib_mrna = ['tmisrib_a_hek', 'tmisrib_b_hek']
tmisrib_ribo = ['ribo_tmisrib_a', 'ribo_tmisrib_b']
isrib_mrna = ['isrib_a_hek', 'isrib_b_hek']
isrib_ribo = ['ribo_isrib_a', 'ribo_isrib_b']

# Output count tables
data_threshold_deseq[untr_ribo + tm_ribo].to_csv(
    deseq_directory + 'tm_ribo_counts.tsv',
    sep='\t')
data_threshold_deseq[untr_ribo + tmisrib_ribo].to_csv(
    deseq_directory + 'tmisrib_ribo_counts.tsv',
    sep='\t')
data_threshold_deseq[untr_ribo + isrib_ribo].to_csv(
    deseq_directory + 'isrib_ribo_counts.tsv',
    sep='\t')

data_threshold_deseq[untr_mrna + tm_mrna].to_csv(
    deseq_directory + 'tm_rna_counts.tsv',
    sep='\t')
data_threshold_deseq[untr_mrna + tmisrib_mrna].to_csv(
    deseq_directory + 'tmisrib_rna_counts.tsv',
    sep='\t')
data_threshold_deseq[untr_mrna + isrib_mrna].to_csv(
    deseq_directory + 'isrib_rna_counts.tsv',
    sep='\t')

data_threshold_deseq[untr_ribo + tm_ribo + untr_mrna + tm_mrna].to_csv(
    deseq_directory + 'tm_counts.tsv',
    sep='\t')
data_threshold_deseq[untr_ribo + tmisrib_ribo + untr_mrna + tmisrib_mrna].to_csv(
    deseq_directory + 'tmisrib_counts.tsv',
    sep='\t')
data_threshold_deseq[untr_ribo + isrib_ribo + untr_mrna + isrib_mrna].to_csv(
    deseq_directory + 'isrib_counts.tsv',
    sep='\t')

#### Read in original count data and process

In [ ]:
# Read in original raw counts from Elife supplement table
original = pd.read_csv(
    str(original),
    sep = '\t',
    index_col = 0)
original = original.drop('size', axis = 1)

# Clean up data
original = original.groupby(level=0).sum() # Combine duplicate named genes
original_rpm = rpm(original)

#### Parse through original data and XPRESSpipe-processed and return dataframes with shared genes only

In [ ]:
# Clean NaNs
data_rpm.shape
original_rpm.shape

data_rpm = data_rpm.dropna()
original_rpm = original_rpm.dropna()

data_rpm.shape
original_rpm.shape

# Make dataframes for each dataset for matching genes
data_genes = data_rpm.index.tolist()
original_genes = original_rpm.index.tolist()
len(data_genes)
len(original_genes)

common_genes = list(set(data_genes).intersection(original_genes))
len(common_genes)

data_common = data_rpm.reindex(index = common_genes)
original_common = original_rpm.reindex(index = common_genes)

#### Over-representation analysis
TopHat2 is .48% more likely to align a read that should be counted as ambiguous
https://www.nature.com/articles/nmeth.4106, supplementary table 5

In [ ]:
tophat_overcount_rate = '0.48%'

#RPF Untr A -> started with ~135,000,000 reads
rpf_start_reads = 135000000 #approx

rpf_test = pd.concat([data_common[['ribo_untr_a']], original_common[['ribo_untr_a']]], axis=1, sort=False)
rpf_test.columns = ['xpresspipe','original']
rpf_test = rpf_test[(rpf_test['xpresspipe'] >= 10) & (rpf_test['original'] >= 10)]

rpf_test.shape
rpf_test.sum()
rpf_log = np.log10(rpf_test+1)

plt.scatter(rpf_log['xpresspipe'].values, rpf_log['original'], s=1)
x, y = [0, 4.9], [-0.05, 5.05]
plt.plot(x, y, c='black')

m = (y[1] - y[0]) / (x[1] - x[0])
b = y[0]

rpf_above = rpf_log.loc[rpf_log['original'] > ((rpf_log['xpresspipe'] * m) + b)]
rpf_above.shape
rpf_above['diff'] = 10**rpf_above['original'] - 10**rpf_above['xpresspipe']
overcounts = rpf_above['diff'].sum()
overcount_rate = overcounts / rpf_start_reads
print('RPF_Untr_A')
print('# Reads overcounted: ' + str(int(round(overcounts))))
print('Overcount rate: ' + str(overcount_rate * 100) + '%')
print('TopHat2 vs STAR overcount ambiguous rate increase: ' + tophat_overcount_rate)

In [ ]:
#RNA Untr A
rna_start_reads = 48000000

rna_test = pd.concat([data_common[['untr_a_hek']], original_common[['untr_a_hek']]], axis=1, sort=False)
rna_test.columns = ['xpresspipe','original']
rna_test.shape
rna_test.sum()

rna_log = np.log10(rna_test+1)

plt.scatter(rna_log['xpresspipe'].values, rna_log['original'], s=1)
x, y = [0, 4.9], [0.18, 5.03]
plt.plot(x, y, c='black')

m = (y[1] - y[0]) / (x[1] - x[0])
b = y[0]

rna_above = rna_log.loc[rna_log['original'] > ((rna_log['xpresspipe'] * m) + b)]
print(rna_above.shape)
rna_above['diff'] = 10**rna_above['original'] - 10**rna_above['xpresspipe']
overcounts = rna_above['diff'].sum()

rna_above['diff'].index.tolist()


plt.scatter(rna_log['xpresspipe'].loc[rna_above['diff'].index.tolist()].values, rna_log['original'].loc[rna_above['diff'].index.tolist()], s=5)
x, y = [0, 4.9], [0.18, 5.03]
plt.plot(x, y, c='red')

len(rna_above['diff'].index.tolist())

overcount_rate = overcounts / rna_start_reads
print('RNA_Untr_A')
print('# Reads overcounted: ' + str(int(round(overcounts))))
print('Overcount rate: ' + str(overcount_rate * 100) + '%')
print('TopHat2 vs STAR overcount ambiguous rate increase: ' + tophat_overcount_rate)

In [ ]:
# Some further analysis
len(rpf_above['diff'].index.tolist())
len(rna_above['diff'].index.tolist())

common_above = list(set(rpf_above['diff'].index.tolist()).intersection(rna_above['diff'].index.tolist()))
len(common_above)

from urllib.request import Request, urlopen

target = "important paralog"
counter = 0
gene_number = 0
for gene in common_above:

    gene_number += 1

    url = "https://www.genecards.org/cgi-bin/carddisp.pl?gene=" + gene
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page = webpage.decode('utf-8').splitlines()

    if any(target in s for s in page) == True:
        counter += 1

print(
    'Number of genes above threshold with annotated paralogs:' 
    + str(counter) 
    + '/' 
    + len(common_above)
)

#### Process internal correlations (within-method)

In [ ]:
# Run correlations between sample alignments
make_internal_correlations(
    data_rpm,
    'internal_correlations_summary_htseq_protein_truncated_v98.png')

make_internal_correlations_supplement(
    data_rpm,
    'internal_correlations_summary_htseq_protein_truncated_v98_all.png')

make_internal_correlations_supplement(
    original_rpm,
    'internal_correlations_summary_ingolia.png')

#### External correlations (between-methods)

In [ ]:
file_list = [
    'isrib_comp_v98_truncated_count_table.tsv',
    'isrib_comp_v98_normal_count_table.tsv',
    'isrib_comp_v98_longest_truncated_count_table.tsv',
    'isrib_comp_v72_normal_count_table.tsv',
    'isrib_comp_v72_longest_truncated_count_table.tsv',
    'isrib_comp_v72_truncated_count_table.tsv']

file_list = [str(comparison_tables_directory) + str(f) for f in file_list]

for file in file_list:

    cycle_external_correlations_supplement(file, original)

make_external_correlations(
    data_common,
    original_common,
    'external_correlations_summary_htseq_protein_truncated_v98.png')

make_external_correlations_supplement(
    data_common,
    original_common,
    'external_correlations_summary_htseq_protein_truncated_v98_all.png')

make_external_correlations_supplement(
    data_common,
    original_common,
    '',
    interactive=True)

make_spearman_pearson_correlations(
    data_common,
    original_common,
    'spearman_pearson_correlations_htseq_protein_truncated_v98.png')

make_paralogs_analysis(
    data_common,
    original_common,
    'highlight_paralogs_comparison_isrib.png')

### Differential Translation Efficiency Analysis

#### Import DESeq2 count data

In [ ]:
# Import count data for sanity checks and DESeq2 output tables
merged_data_XP, merged_data_split_XP, check_data_XP = parse_de_data(
        directory=deseq_directory,
        check_file='ISRIBxpresspipe_thresholded_counts.tsv')

#### Run some sanity checks

In [ ]:
# Some of the genes in plotting cluster on a single y-axis coordinate
# This is due to insufficient RPF counts after RNA thresholding of the data
merged_data_split_XP[
    (merged_data_split_XP['tm_ribo_log2FC'] > 2.7) 
    & (merged_data_split_XP['tm_ribo_log2FC'] < 2.75)
][['tm_ribo_log2FC', 'tm_ribo_padj']]

In [ ]:
# Check fold changes of targets for comparison
print('Raw fold changes:')
print('+++++++++++++++++')
print('ATF4')
print(2**merged_data_split_XP.loc['ATF4']['tm_ribo_log2FC'])
print('=====')
print('ATF5')
print(2**merged_data_split_XP.loc['ATF5']['tm_ribo_log2FC'])
print('=====')
print('PPP1R15A')
print(2**merged_data_split_XP.loc['PPP1R15A']['tm_ribo_log2FC'])
print('=====')
print('DDIT3')
print(2**merged_data_split_XP.loc['DDIT3']['tm_ribo_log2FC'])
print('=====')

print('\nlog2FC comparisons:')
print('+++++++++++++++++++++')
te_sanity(
    data=merged_data_XP[['tm_log2FC', 'tm_padj']],
    check_data=check_data_XP,
    gene_name='ATF4')
print('=====')

te_sanity(
    data=merged_data_XP[['tm_log2FC', 'tm_padj']],
    check_data=check_data_XP,
    gene_name='DDIT3')
print('=====')

te_sanity(
    data=merged_data_XP[['tm_log2FC', 'tm_padj']],
    check_data=check_data_XP,
    gene_name='UCP2')
print('=====')

te_sanity(
    data=merged_data_XP[['tm_log2FC', 'tm_padj']],
    check_data=check_data_XP,
    gene_name='POMGNT1')
print('=====')

te_sanity(
    data=merged_data_XP[['tm_log2FC', 'tm_padj']],
    check_data=check_data_XP,
    gene_name='NDUFA11')

#### Perform pattern analysis on TE data to pull out interesting genes across conditions

In [ ]:
# TE analysis
data_de_plot = merged_data_XP.copy()

data_de_plot['Untreated'] = 0
data_de_plot = data_de_plot[['Untreated','tm_log2FC', 'tmisrib_log2FC','isrib_log2FC']]
data_de_plot.columns = ['Untreated','Tm', 'Tm + ISRIB','ISRIB']

down_strict_1_XP = merged_data_XP.loc[
    (merged_data_XP['tm_log2FC'] - merged_data_XP['tmisrib_log2FC'] <= -1) \
    & (merged_data_XP['tm_padj'] <= 0.001) \
    & (merged_data_XP['tmisrib_padj'] <= 0.001) \
    & ((merged_data_split_XP['tm_ribo_padj'] <= 0.1))]

down_strict_2_XP = merged_data_XP.loc[
    (merged_data_XP['tm_log2FC'] - merged_data_XP['tmisrib_log2FC'] <= -0.58) \
    & (merged_data_split_XP['tm_ribo_log2FC'] <= -0.9) \
    & (merged_data_split_XP['tm_ribo_padj'] <= 0.1)]

down_strict_XP = down_strict_1_XP.index.tolist() + down_strict_2_XP.index.tolist()

# Sanity check count data
print(check_data_XP.loc[down_strict_XP].min().min())

merged_data_XP.loc[down_strict_XP][['tm_log2FC','tm_padj','tmisrib_log2FC','tmisrib_padj']]

#### False-discovery Rate check
- Using the alpha-debt method from dataset decay theory (https://doi.org/10.1101/801696)
- Number of PubMed citations for ISRIB paper == 199
- In random sample of 20 (~10%), only 2 used for some sort of re-analysis
- Will assume a 1/10 use-rate

In [ ]:
n = 199 * (2/20)
alpha = 0.05
from math import ceil
for x in range(0, ceil(n)):
    alpha = alpha / 2
print('Dataset decay alpha: ' + str(alpha))

#### Make riboseq vs RNAseq plots

In [ ]:
# Tm
rp_plot(
    merged_data_split_XP,
    'tm_ribo_log2FC',
    'tm_rna_log2FC',
    'tm_ribo_padj',
    'tm_rna_padj',
    'Tm',
    'Untr',
    merged_data_XP[['tm_log2FC','tm_padj']],
    'tm_padj',
    'Tm_vs_Untr_deseq.png',
    isr,
    uorf_targets,
    down_strict_XP)

# Tm + ISRIB
rp_plot(
    merged_data_split_XP,
    'tmisrib_ribo_log2FC',
    'tmisrib_rna_log2FC',
    'tmisrib_ribo_padj',
    'tmisrib_rna_padj',
    'Tm + ISRIB',
    'Untr',
    merged_data_XP[['tmisrib_log2FC','tmisrib_padj']],
    'tmisrib_padj',
    'TmISRIB_vs_Untr_deseq.png',
    isr,
    uorf_targets,
    down_strict_XP,
    label_up=False,
    label_down=False,
    label_down_custom=False)

# ISRIB
rp_plot(
    merged_data_split_XP,
    'isrib_ribo_log2FC',
    'isrib_rna_log2FC',
    'isrib_ribo_padj',
    'isrib_rna_padj',
    'ISRIB',
    'Untr',
    merged_data_XP[['isrib_log2FC','isrib_padj']],
    'isrib_padj',
    'ISRIB_vs_Untr_deseq.png',
    isr,
    uorf_targets,
    down_strict_XP,
    label_up=False,
    label_down=False,
    label_down_custom=False)

#### Plot TEs

In [ ]:
te_plot(
    data=data_de_plot,
    list_up=isr,
    list_down=down_strict_XP,
    directory=str(deseq_plot_directory),
    plot_all=True)

#### Comparing the methods and their outputs

In [ ]:
"""XPRESSpipe vs Original w/ DESeq1
"""
# Tm
tm_fc_XP = merged_data_split_XP.loc[(merged_data_split_XP['tm_ribo_log2FC'] >= 1) | (merged_data_split_XP['tm_ribo_log2FC'] <= -1)].index.tolist()
tm_te_XP = merged_data_XP.loc[merged_data_XP['tm_padj'] < 0.1].index.tolist()
tm_common_XP = list(set(tm_fc_XP).intersection(tm_te_XP))

venn_plot(
    directory=str(deseq_plot_directory),
    figure_title='Tm',
    gene_list_A=tm_common_XP,
    gene_list_B=ingolia_tm,
    label_A='This paper',
    label_B='Original paper',
    color_A='#d95f02',
    color_B='#1b9e77',
    output_name='tm_deseq2XP_venn.png',
    dpi=3600)

# Tm + ISRIB
tmisrib_fc_XP = merged_data_split_XP.loc[(merged_data_split_XP['tmisrib_ribo_log2FC'] >= 1) | (merged_data_split_XP['tmisrib_ribo_log2FC'] <= -1)].index.tolist()
tmisrib_te_XP = merged_data_XP.loc[merged_data_XP['tmisrib_padj'] < 0.1].index.tolist()
tmisrib_common_XP = list(set(tmisrib_fc_XP).intersection(tmisrib_te_XP))

venn_plot(
    directory=str(deseq_plot_directory),
    figure_title='Tm + ISRIB',
    gene_list_A=tmisrib_common_XP,
    gene_list_B=ingolia_tmisrib,
    label_A='This paper',
    label_B='Original paper',
    color_A='#d95f02',
    color_B='#1b9e77',
    output_name='tm_isrib_deseq2XP_venn.png',
    dpi=3600)

# ISRIB
isrib_fc_XP = merged_data_split_XP.loc[(merged_data_split_XP['isrib_ribo_log2FC'] >= 1) | (merged_data_split_XP['isrib_ribo_log2FC'] <= -1)].index.tolist()
isrib_te_XP = merged_data_XP.loc[merged_data_XP['isrib_padj'] < 0.1].index.tolist()
isrib_common_XP = list(set(isrib_fc_XP).intersection(isrib_te_XP))

venn_plot(
    directory=str(deseq_plot_directory),
    figure_title='ISRIB',
    gene_list_A=isrib_common_XP,
    gene_list_B=ingolia_isrib,
    label_A='This paper',
    label_B='Original paper',
    color_A='#d95f02',
    color_B='#1b9e77',
    output_name='isrib_deseq2XP_venn.png',
    dpi=3600)

"""Original data DESeq2 vs Original data DESeq1
"""
merged_data_OG2, merged_data_split_OG2, check_data_OG2 = parse_de_data(
    directory=dir_og2,
    check_file='ingolia_counts_deduplicated.txt')

# Tm
tm_fc_og2 = merged_data_split_OG2.loc[(merged_data_split_OG2['tm_ribo_log2FC'] >= 1) | (merged_data_split_OG2['tm_ribo_log2FC'] <= -1)].index.tolist()
tm_te_og2 = merged_data_OG2.loc[merged_data_OG2['tm_padj'] < 0.1].index.tolist()
tm_common_og2 = list(set(tm_fc_og2).intersection(tm_te_og2))

venn_plot(
    directory=str(deseq_plot_directory),
    figure_title='Tm',
    gene_list_A=tm_common_og2,
    gene_list_B=ingolia_tm,
    label_A='DESeq2',
    label_B='DESeq1',
    color_A='#7570b3',
    color_B='#1b9e77',
    output_name='tm_deseq2OG_venn.png',
    dpi=3600)

# Tm + ISRIB
tmisrib_fc_og2 = merged_data_split_OG2.loc[(merged_data_split_OG2['tmisrib_ribo_log2FC'] >= 1) | (merged_data_split_OG2['tmisrib_ribo_log2FC'] <= -1)].index.tolist()
tmisrib_te_og2 = merged_data_OG2.loc[merged_data_OG2['tmisrib_padj'] < 0.1].index.tolist()
tmisrib_common_og2 = list(set(tmisrib_fc_og2).intersection(tmisrib_te_og2))

venn_plot(
    directory=str(deseq_plot_directory),
    figure_title='Tm + ISRIB',
    gene_list_A=tmisrib_common_og2,
    gene_list_B=ingolia_tmisrib,
    label_A='DESeq2',
    label_B='DESeq1',
    color_A='#7570b3',
    color_B='#1b9e77',
    output_name='tm_isrib_deseq2OG_venn.png',
    dpi=3600)

# ISRIB
isrib_fc_og2 = merged_data_split_OG2.loc[(merged_data_split_OG2['isrib_ribo_log2FC'] >= 1) | (merged_data_split_OG2['isrib_ribo_log2FC'] <= -1)].index.tolist()
isrib_te_og2 = merged_data_OG2.loc[merged_data_OG2['isrib_padj'] < 0.1].index.tolist()
isrib_common_og2 = list(set(isrib_fc_og2).intersection(isrib_te_og2))

venn_plot(
    directory=str(deseq_plot_directory),
    figure_title='ISRIB',
    gene_list_A=isrib_common_og2,
    gene_list_B=ingolia_isrib,
    label_A='DESeq2',
    label_B='DESeq1',
    color_A='#7570b3',
    color_B='#1b9e77',
    output_name='isrib_deseq2OG_venn.png',
    dpi=3600)

"""XPRESSpipe DESeq2 vs Original w/ DESeq2
"""
# Tm
venn_plot(
    directory=str(deseq_plot_directory),
    figure_title='Tm',
    gene_list_A=tm_common_XP,
    gene_list_B=tm_common_og2,
    label_A='This paper',
    label_B='Original paper',
    color_A='#d95f02',
    color_B='#7570b3',
    output_name='tm_deseq2both_venn.png',
    dpi=3600)

# Tm + ISRIB
venn_plot(
    directory=str(deseq_plot_directory),
    figure_title='Tm + ISRIB',
    gene_list_A=tmisrib_common_XP,
    gene_list_B=tmisrib_common_og2,
    label_A='This paper',
    label_B='Original paper',
    color_A='#d95f02',
    color_B='#7570b3',
    output_name='tm_isrib_deseq2both_venn.png',
    dpi=3600)

# ISRIB
venn_plot(
    directory=str(deseq_plot_directory),
    figure_title='ISRIB',
    gene_list_A=isrib_common_XP,
    gene_list_B=isrib_common_og2,
    label_A='This paper',
    label_B='Original paper',
    color_A='#d95f02',
    color_B='#7570b3',
    output_name='isrib_deseq2both_venn.png',
    dpi=3600)